Vamos a meter la variable head_to_head desde 2021 hasta 2023

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from ranking_scores import puntos_ranking

In [2]:
def calculate_head_to_head(dataset):
    # Convertir la columna 'tourney_date' a formato de fecha
    dataset['tourney_date'] = pd.to_datetime(dataset['tourney_date'], format='%Y%m%d')

    # Ordenar el dataset por fecha
    dataset = dataset.sort_values('tourney_date')

    # Inicializar una nueva columna para el historial de enfrentamientos directos
    dataset['head_to_head'] = 0.5
    '''dataset['num_matches'] = 0'''
    # Iterar sobre las filas del dataset
    for index, row in dataset.iterrows():
        # Obtener los partidos anteriores entre los dos jugadores
        previous_matches = dataset[((dataset['winner_id'] == row['winner_id']) & (dataset['loser_id'] == row['loser_id']) | (dataset['winner_id'] == row['loser_id']) & (dataset['loser_id'] == row['winner_id'])) & (dataset['tourney_date'] < row['tourney_date'])]

        if not previous_matches.empty:
            # Calcular el porcentaje de veces que el 'winner_id' ha ganado
            wins = len(previous_matches[previous_matches['winner_id'] == row['winner_id']])
            total = len(previous_matches)
            dataset.at[index, 'head_to_head'] = wins / total
            '''# Calcular el número de partidos jugados
            dataset.at[index, 'num_matches'] = total'''

    return dataset

In [3]:
# Cargar los datos de los años 2022 y 2023
data_2021 = pd.read_csv('atp_matches_2021.csv')
data_2022 = pd.read_csv('atp_matches_2022.csv')
data_2023 = pd.read_csv('atp_matches_2023.csv')

# Agregar una columna 'source' a cada DataFrame
data_2021['source'] = 'atp_matches_2021.csv'
data_2022['source'] = 'atp_matches_2022.csv'
data_2023['source'] = 'atp_matches_2023.csv'

# Combinar los datos en un solo DataFrame
data_all = pd.concat([data_2021, data_2022, data_2023])

# Aplicar la función calculate_head_to_head
data_all = calculate_head_to_head(data_all)

# Filtrar los datos para obtener solo los partidos de 2023 del archivo 'atp_matches_2023.csv'
dataset = data_all[data_all['source'] == 'atp_matches_2023.csv']

In [4]:
dataset_r = pd.read_csv('atp_rankings_20s.csv')

### Realizamos el tratamiento de los NaN's antes de nada

In [5]:
# Encontramos el valor más común entre 'Grass', 'Hard', y 'Clay'
most_common = dataset[dataset['surface'].isin(['Grass', 'Hard', 'Clay'])]['surface'].mode()[0]

# Reemplazamos los valores que no son 'Grass', 'Hard', o 'Clay' (los NaN's) con el valor más común
dataset.loc[~dataset['surface'].isin(['Grass', 'Hard', 'Clay']), 'surface'] = most_common

In [6]:
pk = puntos_ranking(dataset, dataset_r, 2023)

c:\Users\Guille\OneDrive\Escritorio\Deepty Projects\ranking_scores.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['tourney_date'] = pd.to_datetime(dataset['tourney_date'], format='%Y%m%d')
c:\Users\Guille\OneDrive\Escritorio\Deepty Projects\ranking_scores.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['year_month'] = dataset['tourney_date'].dt.to_period('M')


In [7]:
dataset.loc[:, 'winner_rank_points'], dataset.loc[:, 'loser_rank_points'] = pk['winner_points'], pk['loser_points']

In [8]:
dataset.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,source,head_to_head,year_month
69,2023-2843,Adelaide 1,Hard,32,A,2023-01-02,279,207733,NaN,NaN,...,8.0,7.0,12.0,42.0,990.0,83.0,630.0,atp_matches_2023.csv,0.5,2023-01
70,2023-2843,Adelaide 1,Hard,32,A,2023-01-02,278,111575,8.0,NaN,...,9.0,8.0,12.0,20.0,1990.0,57.0,813.0,atp_matches_2023.csv,0.5,2023-01
71,2023-2843,Adelaide 1,Hard,32,A,2023-01-02,277,206173,6.0,NaN,...,8.0,1.0,4.0,15.0,2410.0,582.0,52.0,atp_matches_2023.csv,1.0,2023-01
72,2023-2843,Adelaide 1,Hard,32,A,2023-01-02,276,106423,NaN,WC,...,12.0,5.0,6.0,93.0,592.0,34.0,1163.0,atp_matches_2023.csv,0.5,2023-01
76,2023-2843,Adelaide 1,Hard,32,A,2023-01-02,272,111456,NaN,NaN,...,5.0,0.0,2.0,63.0,775.0,67.0,741.0,atp_matches_2023.csv,0.5,2023-01


In [9]:
'''Indice 7: ganador del partido.
Indice 15: perdedor del partido.
Indice -1: puntos en el ranking del jugador ganador
Indice -3: puntos en el ranking del jugador perdedor
Indice 2: tipo de campo del partido (grass, hard o clay)'''
'''X = dataset.iloc[:, [7, 15, -1, -3, 2]].values
y = dataset.iloc[:, 7].values'''

'X = dataset.iloc[:, [7, 15, -1, -3, 2]].values\ny = dataset.iloc[:, 7].values'

In [10]:
columnas_X = ['winner_id', 'loser_id', 'winner_rank_points', 'loser_rank_points', 'head_to_head', 'surface']
columna_y = 'winner_id'
X = dataset[columnas_X].values
y = dataset[columna_y].values


In [11]:
X[:, 0] = X[:, 0].astype(str)
X[:, 1] = X[:, 1].astype(str)
y = y.astype(str)

In [12]:
# Crear el codificador
encoder = OneHotEncoder(sparse_output=False)

# Ajustar el codificador con los datos de ambos jugadores y 'y'
encoder.fit(X[:, [0, 1]].reshape(-1, 1))

# Transformar las columnas de los jugadores
player1_onehot = encoder.transform(X[:, 0].reshape(-1, 1))
player2_onehot = encoder.transform(X[:, 1].reshape(-1, 1))

# Transformar el vector objetivo 'y' con el mismo codificador 'X'
y = encoder.transform(y.reshape(-1, 1))

# Concatenar las columnas one-hot con el resto de tus datos
X = np.concatenate([player1_onehot, player2_onehot, X[:, 2:]], axis=1)


In [13]:
# Matriz de booleanos aleatorios del mismo tamaño que tus datos
swap = np.random.rand(len(X)) > 0.5

# Intercambiamos las posiciones de los jugadores en 'X' donde 'swap' es True
X[swap, :440], X[swap, 440:880] = X[swap, 440:880], X[swap, :440].copy()

# Intercambiamos las posiciones de los puntos en el ranking donde 'swap' es True
X[swap, 880], X[swap, 881] = X[swap, 881], X[swap, 880].copy()

mask = (swap) & (X[:, -2] != -1)
X[mask, -2] = 1 - X[mask, -2]

In [14]:
# Creamos el escalador
scaler = MinMaxScaler()

# Ajustamos y transformamos las dos últimas columnas de tus datos
X[:, [-4, -3]] = scaler.fit_transform(X[:, [-4, -3]])

In [15]:
# Esto es para el tipo de campo
# Crear el codificador
encoder2 = OneHotEncoder(sparse_output=False)

# Ajustar y transformar la columna del tipo de campo
field_type_onehot = encoder2.fit_transform(X[:, 883].reshape(-1, 1))

# Concatenar la columna one-hot con el resto de los datos
X = np.concatenate([X[:, :883], field_type_onehot], axis=1)

In [16]:
from sklearn.model_selection import train_test_split

# Luego, dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
X_train = X_train.astype('float64')
y_train = y_train.astype('float64')

In [18]:
# Definimos el modelo
model = Sequential()

# Añadir la capa de entrada y la primera capa oculta
model.add(Dense(units=128, activation='relu', input_dim=X_train.shape[1]))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
# Añadir la segunda capa oculta
# model.add(Dense(units=32, activation='tanh'))

# Añadir la capa de salida
model.add(Dense(units=y.shape[1], activation='softmax'))

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=25, batch_size=32)

Epoch 1/25
75/75 [==============================] - 1s 2ms/step - loss: 5.7798 - accuracy: 0.0578
Epoch 2/25
75/75 [==============================] - 0s 2ms/step - loss: 4.7449 - accuracy: 0.0678
Epoch 3/25
75/75 [==============================] - 0s 2ms/step - loss: 4.3128 - accuracy: 0.2115
Epoch 4/25
75/75 [==============================] - 0s 2ms/step - loss: 3.8679 - accuracy: 0.3798
Epoch 5/25
75/75 [==============================] - 0s 2ms/step - loss: 3.3692 - accuracy: 0.5335
Epoch 6/25
75/75 [==============================] - 0s 2ms/step - loss: 2.8648 - accuracy: 0.5988
Epoch 7/25
75/75 [==============================] - 0s 2ms/step - loss: 2.4015 - accuracy: 0.6420
Epoch 8/25
75/75 [==============================] - 0s 2ms/step - loss: 2.0016 - accuracy: 0.6709
Epoch 9/25
75/75 [==============================] - 0s 2ms/step - loss: 1.6687 - accuracy: 0.7052
Epoch 10/25
75/75 [==============================] - 0s 2ms/step - loss: 1.3962 - accuracy: 0.7412
Epoch 11/25
75/75 [

In [19]:
X_test = X_test.astype('float64')
y_test = y_test.astype('float64')

In [20]:
# Hacemos la predicción
prediccion_test = model.predict(X_test)

# Obtenemos los índices donde hay un 1 en las primeras 440 columnas y en las siguientes 440 columnas
indices_1 = np.where(X_test[:, :440] == 1)[1]
indices_2 = np.where(X_test[:, 440:880] == 1)[1]

# Obtenemos las predicciones para los índices obtenidos
predicciones_1 = prediccion_test[np.arange(len(prediccion_test)), indices_1]
predicciones_2 = prediccion_test[np.arange(len(prediccion_test)), indices_2]

# Encontramos el índice del máximo para cada fila
max_indices = np.where(predicciones_1 > predicciones_2, indices_1, indices_2)

# Convertimos max_indices en un array 2D
max_indices_2D = np.eye(encoder.categories_[0].shape[0])[max_indices]

# Usamos el codificador para transformar los índices a sus etiquetas originales
etiquetas_predichas = encoder.inverse_transform(max_indices_2D)

print(etiquetas_predichas)

19/19 [==============================] - 0s 1ms/step
[['126205']
 ['104527']
 ['200670']
 ['207733']
 ['207733']
 ['210097']
 ['126094']
 ['200615']
 ['105526']
 ['104925']
 ['126128']
 ['104792']
 ['104918']
 ['149285']
 ['144985']
 ['126774']
 ['106398']
 ['106234']
 ['111797']
 ['111190']
 ['105138']
 ['111456']
 ['206173']
 ['209098']
 ['132283']
 ['104925']
 ['134770']
 ['126774']
 ['111460']
 ['111815']
 ['126203']
 ['207518']
 ['207733']
 ['111456']
 ['200624']
 ['126203']
 ['131951']
 ['144869']
 ['111153']
 ['105173']
 ['200005']
 ['207733']
 ['202104']
 ['126845']
 ['104925']
 ['111456']
 ['206173']
 ['207686']
 ['100644']
 ['111575']
 ['134770']
 ['126207']
 ['209414']
 ['134868']
 ['136440']
 ['111456']
 ['144869']
 ['209414']
 ['111797']
 ['206173']
 ['200175']
 ['111456']
 ['122330']
 ['207518']
 ['134868']
 ['207686']
 ['104792']
 ['111153']
 ['208029']
 ['111460']
 ['209976']
 ['127157']
 ['206173']
 ['136440']
 ['106421']
 ['123755']
 ['126094']
 ['208014']
 ['111581']

In [21]:
# Usamos el codificador para transformar las etiquetas codificadas a sus etiquetas originales
etiqueta_y_test = encoder.inverse_transform(y_test)

print(etiqueta_y_test)

[['207733']
 ['104527']
 ['200670']
 ['200175']
 ['207733']
 ['111460']
 ['126094']
 ['200615']
 ['105526']
 ['104925']
 ['106045']
 ['104792']
 ['104918']
 ['149285']
 ['144985']
 ['126774']
 ['106398']
 ['106234']
 ['111797']
 ['106233']
 ['105138']
 ['111456']
 ['206173']
 ['209098']
 ['111797']
 ['104925']
 ['134770']
 ['126774']
 ['111460']
 ['111815']
 ['126203']
 ['207518']
 ['104745']
 ['111456']
 ['200624']
 ['126205']
 ['131951']
 ['144869']
 ['111153']
 ['200005']
 ['126094']
 ['202103']
 ['202104']
 ['126845']
 ['208029']
 ['111456']
 ['206173']
 ['207686']
 ['106421']
 ['111575']
 ['209950']
 ['126207']
 ['200221']
 ['134868']
 ['106214']
 ['200221']
 ['104918']
 ['209414']
 ['111797']
 ['206173']
 ['200175']
 ['111456']
 ['106233']
 ['209113']
 ['126207']
 ['207686']
 ['202103']
 ['111153']
 ['208029']
 ['105676']
 ['202261']
 ['127157']
 ['206173']
 ['136440']
 ['206173']
 ['123755']
 ['126094']
 ['124186']
 ['111581']
 ['200175']
 ['200436']
 ['104925']
 ['122669']
 ['1

In [22]:
# Veamos cuántos partidos ha acertado para el test
aciertos = 0
for i in range(len(etiquetas_predichas)):
    if etiquetas_predichas[i] == etiqueta_y_test[i]:
        aciertos += 1
print(f"Precisión del test: {np.round(aciertos / len(etiquetas_predichas)*100, 2)}%")

Precisión del test: 65.55%


In [23]:
# Dividimos X_test en dos partes
X_test_1 = X_test[:, :440]
X_test_2 = X_test[:, 440:880]

# Aplicamos inverse_transform a cada parte
jugadores_test_1 = encoder.inverse_transform(X_test_1).ravel()
jugadores_test_2 = encoder.inverse_transform(X_test_2).ravel()

# Verificamos si cada etiqueta predicha está en los jugadores en juego
ganadores_en_juego_1 = np.isin(etiquetas_predichas, jugadores_test_1)
ganadores_en_juego_2 = np.isin(etiquetas_predichas, jugadores_test_2)

# Imprimimos los casos donde el ganador predicho no está en el juego
print(f"Ganadores predichos que no están en el juego:", len(np.array(etiquetas_predichas)[~ganadores_en_juego_1]) + len(np.array(etiquetas_predichas)[~ganadores_en_juego_2]))

Ganadores predichos que no están en el juego: 35
